In [3]:
from elasticsearch import Elasticsearch

In [8]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","vfkWRFChlFKFBDPz*0y8"),
    ca_certs="C:/Users/Abhishek_Jaiswal/Downloads/elasticsearch-8.13.4/config/certs/http_ca.crt"
)
es.ping()

True

## Prepare the data

In [4]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige


In [9]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [10]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [13]:
df.head(2)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645782, -0.002634167, -0.0035884539, 0.0..."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660744, -0.028755417, -0.02033243, 0.03..."


In [15]:
df.shape

(500, 9)

In [14]:
es.ping()

True

## Create new index in ElasticSearch!

In [16]:
indexMapping = {
    "properties":{
        "ProductID":{
            "type":"long"
        },
        "ProductName":{
            "type":"text"
        },
        "ProductBrand":{
            "type":"text"
        },
        "Gender":{
            "type":"text"
        },
        "Price (INR)":{
            "type":"long"
        },
        "NumImages":{
            "type":"long"
        },
        "Description":{
            "type":"text"
        },
        "PrimaryColor":{
            "type":"text"
        },
        "DescriptionVector":{
            "type":"dense_vector",
            "dims": 768,
            "index":True,
            "similarity": "l2_norm"
        }

    }
}

In [17]:
from indexMapping import indexMapping

es.indices.create(index="all_products", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into index

In [18]:
record_list = df.to_dict("records")
record_list[0]

{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black',
 'DescriptionVector': array([ 2.76457816e-02, -2.63416697e-03, -3.58845387e-03,  5.13587482e-02,
         3.09660845e-02,  1.40506886e-02,  7.27059599e-03,  3.13870944e-02,
        -6.23787642e-02, -3.82879004e-03,  3.15214284e-02,  7.55472854e-02,
         2.12642434e-03,  4.64894436e-02,  5.07448576e-02, -1.71942525e-02,
         1.22891665e-02, -1.95682663e-02, -9.6

In [19]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [20]:
es.count(index="all_products")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [21]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

C:\Users\Abhishek_Jaiswal\AppData\Local\Temp\ipykernel_8856\2784084207.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.61429423,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.61429423,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]